In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import psana as ps
%matplotlib inline

In [ ]:
from h5py import File
from matplotlib.colors import LogNorm
from scipy.ndimage.filters import gaussian_filter

In [ ]:
from sys import path
path.append('/cds/home/e/elio/modules/')
from surf import surf

In [ ]:
run = 185
vers = 0
raw = File("/reg/d/psdm/tmo/tmolv2918/scratch/elio/v%d/run%d_v%d.h5" % (vers, run, vers), "r")
raw['Nbad'][()], raw['Nfound'][()]

In [ ]:
run = 231
vers = 6
raw = File("/reg/d/psdm/tmo/tmolv2918/scratch/preproc/v%d/run%d_v%d.h5" % (vers, run, vers), "r")
raw['Nbad'][()], raw['Nfound'][()]

In [ ]:


order = raw['timestamp'][:].argsort()
timestamps = raw['timestamp'][:][order]
ts = (timestamps - timestamps.min()) * 1e-6 # in ms
#wfs = raw['iToF_wf'][:][order]

tflight = np.arange(13072) * 10 / 60 / 100
t0 = 0.190
A = (13.211 - t0)/np.sqrt(100)
m_q = ((tflight - t0)/A)**2
#mpks = np.array([12, 13, 14, 15, 27, 29, 31, 39, 41, 42, 43, 85, 100])

#Ipk = np.zeros((len(mpks), len(order)))
#for i, mpk, in enumerate(mpks):
#    Ipk[i] = wfs[:,(mpk - 0.25 < m_q) & (m_q < mpk + 0.25)].mean(1)
mpks = raw['mpks'][:]
Ipk = raw['Ipk'][:][order]

gmd = raw['gmd_energy'][:][order]
xgmd = raw['xgmd_energy'][:][order]
l3 = raw['ebeam_ebeamL3Energy'][:][order]

even = raw['evrs'][:,41].astype(bool)[order]
jet = raw['evrs'][:,70].astype(bool)[order]
xray = ~raw['evrs'][:,161].astype(bool)[order]
goose = raw['evrs'][:,68].astype(bool)[order]

delay = raw['epics_las_fs14_target_time'][:][order]
delays, delay_idx = np.unique(np.around(delay, 4), return_inverse=True)


vls = raw['vls'][:][order]
badvls = vls.mean(1) < -9000
vls_ = gaussian_filter(vls, (0,0))
vls_ -= np.median(vls_[~badvls].mean(0))
vls_[vls_< 50] = 0
CoM = (vls_ * np.arange(2048)).mean(1) / vls_.mean(1)

good = (~np.isnan(CoM)) & (gmd > -0.5)

In [ ]:

plt.plot(np.diff(timestamps[badvls])*1e-9)

In [ ]:

plt.plot(vls_[:10000:1000].T)
plt.plot(vls_.mean(0), 'k')
#plt.xlim(1100, 1500)
vls_low, vls_high = 1100, 1450
plt.xlim(vls_low, vls_high)

In [ ]:
plt.hist(gmd, bins=np.linspace(-0.1, 0.4, 300));

In [ ]:
#a, b, c = plt.hist(vls_[:,1100:1500].mean(1), bins=200);
#plt.hist(vls_[xray,1100:1500].mean(1), bins=b, histtype='step');
plt.hist(vls_[~xray,1100:1500].mean(1), bins=1000, histtype='step');

In [ ]:
mi = 43

vls_low, vls_high = 0, 450
plt.figure(figsize=(15,10))
plt.subplot(3,2,1)
plt.hist2d(l3[good], gmd[good], bins=200);
plt.xlabel("L3")
plt.ylabel("GMD")

plt.subplot(3,2,2)
plt.hist2d(l3[good], vls[good,vls_low:vls_high].mean(1), bins=200);
plt.xlabel("L3")
plt.ylabel("VLS int.")

plt.subplot(3,2,3)
plt.hist2d(gmd[good], vls[good,vls_low:vls_high].mean(1), bins=200);
plt.xlabel("GMD")
plt.ylabel("VLS int.")

plt.subplot(3,2,4)
plt.hist2d(l3[good&jet], Ipk[good&jet, np.abs(mpks-mi).argmin()], bins=200);
plt.xlabel("L3")
plt.ylabel("Mass 43 int.")

plt.subplot(3,2,5)
plt.hist2d(vls[good&jet, vls_low:vls_high].mean(1), Ipk[good&jet, np.abs(mpks-mi).argmin()], bins=200);
plt.xlabel("VLS int.")
plt.ylabel("Mass 43 int.")

plt.subplot(3,2,6)
plt.hist2d(gmd[good&jet], Ipk[good&jet, np.abs(mpks-mi).argmin()], bins=200);
plt.xlabel("GMD")
plt.ylabel("Mass 43 int.")
#plt.xlim(1200, 1500)
plt.tight_layout()

In [ ]:
plt.hist2d(l3[good], vls[good,1100:1450].mean(1), bins=200);
#plt.xlim(1100, 1500)
#plt.ylim(304, 360)

In [ ]:
vls.shape

In [ ]:
plt.hist2d(gmd[good], vls[good,1100:1450].mean(1), bins=200);

In [ ]:
plt.hist2d(CoM[good], l3[good], bins=200);

In [ ]:
plt.hist2d(vls[good, 1100:1450].mean(1), Ipk[np.abs(mpks-43).argmin(), good], bins=50);

In [ ]:
plt.hist2d(gmd[good&jet], Ipk[np.abs(mpks-43).argmin(), good&jet], bins=50);

In [ ]:
plt.hist2d(l3[good&jet], Ipk[good&jet, np.abs(mpks-43).argmin()], bins=50);

In [ ]:
plt.figure(figsize=(20,4))
index = 1
nbins = 80
filt =   (5690 < l3) & ~even
#filt = (np.arange(len(gmd)) > len(gmd)*0.8) #& (1320 < CoM)
for off in [3, 2, 1]:
    plt.subplot(1,7,index)
    index+=1
    x, y = CoM[:-off][filt[off:]], l3[off:][filt[off:]]
    x, y = x[:min(len(x), len(y))], y[:min(len(x), len(y))]
    x, y = x[~np.isnan(x)], y[~np.isnan(x)]
    plt.hist2d(x, y, bins=(np.linspace(1250, 1450, nbins), np.linspace(5660, 5740, nbins)));
    plt.title("offset: %d, R=%.4f" % (-off, np.corrcoef(x, y)[0,1]))
plt.subplot(1,7,index)
index+=1
x, y = CoM[filt], l3[filt]
x, y = x[:min(len(x), len(y))], y[:min(len(x), len(y))]
x, y = x[~np.isnan(x)], y[~np.isnan(x)]
plt.hist2d(x, y, bins=(np.linspace(1250, 1450, nbins), np.linspace(5660, 5740, nbins)));
plt.title("offset: %d, R= %.4f" % (0, np.corrcoef(x, y)[0,1]))
for off in [1, 2, 3]:
    plt.subplot(1,7,index)
    index+=1
    x, y = CoM[off:][filt[:-off]], l3[:-off][filt[:-off]]
    x, y = x[:min(len(x), len(y))], y[:min(len(x), len(y))]
    x, y = x[~np.isnan(x)], y[~np.isnan(x)]
    plt.hist2d(x, y, bins=(np.linspace(1250, 1450, nbins), np.linspace(5660, 5740, nbins)));
    plt.title("offset: %d, R= %.4f" % (off, np.corrcoef(x, y)[0,1]))
plt.tight_layout()

In [ ]:
plt.figure(figsize=(10,4))
index = 1
nbins = 180
filt =   (5690 < l3) & ~even
#filt = (np.arange(len(gmd)) > len(gmd)*0.8) #& (1320 < CoM)
for off in [3, 2, 1]:
    #plt.subplot(1,7,index)
    index+=1
    x, y = CoM[:-off][filt[off:]], l3[off:][filt[off:]]
    x, y = x[:min(len(x), len(y))], y[:min(len(x), len(y))]
    x, y = x[~np.isnan(x)], y[~np.isnan(x)]
    plt.hist(np.polyval( np.polyfit(x, y, 1), x) - y, bins=np.linspace(-25, 25, nbins), histtype='step', label=off);
    plt.title("offset: %d, R=%.4f" % (-off, np.corrcoef(x, y)[0,1]))
#plt.subplot(1,7,index)
index+=1
x, y = CoM[filt], l3[filt]
x, y = x[:min(len(x), len(y))], y[:min(len(x), len(y))]
x, y = x[~np.isnan(x)], y[~np.isnan(x)]
plt.hist(np.polyval( np.polyfit(x, y, 1), x) - y, bins=np.linspace(-25, 25, nbins), histtype='step', label=0);
#plt.title("offset: %d, R= %.4f" % (0, np.corrcoef(x, y)[0,1]))
for off in [1, 2, 3]:
    #plt.subplot(1,7,index)
    index+=1
    x, y = CoM[off:][filt[:-off]], l3[:-off][filt[:-off]]
    x, y = x[:min(len(x), len(y))], y[:min(len(x), len(y))]
    x, y = x[~np.isnan(x)], y[~np.isnan(x)]
    plt.hist(np.polyval( np.polyfit(x, y, 1), x) - y, bins=np.linspace(-25, 25, nbins), histtype='step', label=off);
    #plt.title("offset: %d, R= %.4f" % (off, np.corrcoef(x, y)[0,1]))
plt.tight_layout()
plt.legend()

In [ ]:
plt.figure(figsize=(15,4))
index = 1
nbins = 180
filt =   even | ~even
#filt = (np.arange(len(gmd)) > len(gmd)*0.8) #& (1320 < CoM)
resids = []
for off in [-3, -2, -1, 0, 1, 2, 3]:
    #plt.subplot(1,7,index)
    index+=1
    t, x, y = ts[10:-10][filt[10:-10]], CoM[10+off:-10+off][filt[10:-10]], l3[10:-10][filt[10:-10]]
    t, x, y = t[:min(len(x), len(y))], x[:min(len(x), len(y))], y[:min(len(x), len(y))]
    t, x, y = t[~np.isnan(x)], x[~np.isnan(x)], y[~np.isnan(x)]
    plt.plot(t*1e-3, gaussian_filter(np.abs(np.polyval( np.polyfit(x, y, 1), x) - y), 50), label=off);
    resids.append(np.polyval( np.polyfit(x, y, 1), x) - y)

plt.legend()
#plt.xlim(0, 500)
plt.ylim(0, 7)
plt.grid()
plt.gca().twinx()
plt.plot(ts*1e-3, gaussian_filter(l3, 50), 'k')
plt.tight_layout()

In [ ]:
plt.figure(dpi=120)
for offset in [-3, -2, -1, 0, 1, 2, 3]:
    plt.hist(vls[10+offset:-10+offset, 1000:1500].mean(1)[(~xray)[10:-10]], bins=np.linspace(298, 385, 120), histtype='step', label=offset)
#plt.hist(vls.mean(1)[xray], bins=100, histtype='step', density=True)
#plt.yscale('log')
plt.hist(vls[10:-10, 500:1000].mean(1)[(~xray)[10:-10]] , bins=np.linspace(298, 385, 120), histtype='step', color='k')
plt.legend()
#plt.ylim(0, 800)
plt.yscale('log')

In [ ]:
plt.hist(gmd*1e3, bins = np.linspace(-10, 300, 100));
plt.yscale('log')

In [ ]:
plt.hist2d(gmd*1e3, )

In [ ]:
plt.figure(dpi=200)
plt.hist2d(vls[10:-10, 1000:1500].mean(1)[(~xray)[10:-10]], vls[10-1:-10-1, 1000:1500].mean(1)[(~xray)[10:-10]], bins=np.linspace(298, 385, 170), norm=LogNorm());
plt.axis('square')

In [ ]:
plt.hist2d(vls[10:-10, 1000:1500].mean(1)[(~xray)[10:-10]], vls[10-2:-10-2, 1000:1500].mean(1)[(~xray)[10:-10]], bins=np.linspace(298, 385, 170), norm=LogNorm());
plt.axis('square')

In [ ]:
plt.hist2d(vls[10:-10, 1000:1500].mean(1)[(~xray)[10:-10]], vls[10+1:-10+1, 1000:1500].mean(1)[(~xray)[10:-10]], bins=np.linspace(298, 385, 170), norm=LogNorm());

In [ ]:
plt.figure(dpi=200)
plt.hist2d(vls[10:-10, 1000:1500].mean(1)[(~xray)[10:-10]]-vls[10:-10, 500:1000].mean(1)[(~xray)[10:-10]], 300+vls[10-1:-10-1, 1000:1500].mean(1)[(~xray)[10:-10]]-vls[10-1:-10-1, 500:1000].mean(1)[(~xray)[10:-10]], bins=np.linspace(298, 385, 170), norm=LogNorm());
#plt.xlim(295, 310)
#plt.ylim(295, 310)
plt.axis("square")

In [ ]:
plt.hist(vls[10:-10, 500:1000].mean(1)[(~xray)[10:-10]], bins=np.linspace(298, 385, 170));
plt.yscale('log')

In [ ]:
plt.figure(figsize=(25,4))
data= []
for offset in [-3, -2, -1, 0, 1, 2, 3]:
    plt.plot(vls[10+offset:-10+offset,1000:1500].mean(1)[~xray[10:-10]] + offset*0, label=offset, alpha=1)
    data.append(vls[10+offset:-10+offset,1000:1500].mean(1)[~xray[10:-10]] - vls[10+offset:-10+offset,0:500].mean(1)[~xray[10:-10]])
#plt.hist(vls.mean(1)[xray], bins=100, histtype='step', density=True)
#plt.yscale('log')
plt.legend()
#plt.xlim(0,600)
plt.xlim(0, 2000)
data = np.array(data)

In [ ]:
badvls_where = np.where(badvls)[0]
bykik_where = np.where(~xray)[0]
lim = 4
ln = len(xray)
badvls_where = badvls_where[(lim<=badvls_where)&(badvls_where<ln-lim)]
bykik_where = bykik_where[(lim<=bykik_where)&(bykik_where<ln-lim)]
print(np.unique(np.diff(badvls_where), return_counts=True))
print(np.unique(np.diff(bykik_where), return_counts=True))

In [ ]:
offsets = [-1, 0, 1]
data = np.zeros((len(bykik_where), len(offsets)))
for i, idx in enumerate(bykik_where):
    for j, offset in enumerate(offsets):
        offset_idx = idx + offset
        if offset_idx in badvls_where:
            data[i, j] = -9999
        else:
            data[i, j] = vls[offset_idx, 1000:1500].mean() - vls[offset_idx, 250:750].mean()

In [ ]:
data.shape, len(bykik_where)

In [ ]:
plt.hist2d(gmd, vls[:, 1000:1500].mean(1) - vls[:, 250:750].mean(1), bins=(np.linspace(-0.1, 0.5, 150), np.linspace(-5, 80, 100)), norm=LogNorm());

In [ ]:
v_thresh = 1.5 #check this each run??
E_thresh = 0.05 #check this each run??
offsets = np.zeros_like(bykik_where)
types = np.zeros_like(bykik_where)

found = 0
for i, idx in enumerate(bykik_where):
    
    vs = vls[idx-1:idx+2, 1000:1500].mean(1) - vls[idx-1:idx+2, 250:750].mean(1) - 20 * badvls[idx-1:idx+2]
    vs = np.digitize(vs, bins=[-99999, -15, v_thresh, 99999]) - 2 # -1: badvls, 0: lowvls, +1: highvls
    
    Es = gmd[idx-2:idx+3][[0, 1, 3, 4]] > E_thresh
    
    if np.all(vs==[1,0,1]) and np.all(Es): # situation 1, checked
        offsets[i] = 0
        types[i] = 1
    elif np.all(vs==[ 1,-1, 1]) and np.all(Es): # situation 2, didn't find any yet
        offsets[i] = 0
        types[i] = 2
    elif np.all(vs==[ 1, 0,-1]) and np.all(Es): # situation 3, didn't find any yet
        offsets[i] = 0
        types[i] = 3
    elif np.all(vs==[-1, 0, 1]) and np.all(Es): # situation 4, checked
        offsets[i] = 0
        types[i] = 4
    elif np.all(vs==[ 1, 1, 0]) and np.all(Es): # situation 5, didn't find any this run (check 187)
        offsets[i] = 1
        types[i] = 5
    elif np.all(vs==[ 1, 1,-1]) and np.all(Es): # situation 6, didn't find any this run (check 187, but probably rare)
        offsets[i] = 1
        types[i] = 6
    elif np.all(vs==[ 1,-1, 0]) and np.all(Es): # situation 7, didn't find any this run (check 187, but probably rare)
        offsets[i] = 1
        types[i] = 7
    elif np.all(vs==[-1, 1, 0]) and np.all(Es): # situation 8, didn't find any this run (check 187, but probably rare)
        offsets[i] = 1
        types[i] = 8
    elif np.all(vs==[ 0, 1, 1]) and np.all(Es): # situation 9, checked
        offsets[i] = -1
        types[i] = 9
    elif np.all(vs==[-1, 1, 1]) and np.all(Es): # situation 10, didn't find any yet
        offsets[i] = -1
        types[i] = 10
    elif np.all(vs==[ 0,-1, 1]) and np.all(Es): # situation 11, checked
        offsets[i] = -1
        types[i] = 11
    elif np.all(vs==[ 0, 1,-1]) and np.all(Es): # situation 12, checked
        offsets[i] = -1
        types[i] = 12
    elif np.all(vs==[ 1, 0, 0]) and np.all(Es==[1, 1, 0, 1]): # situation 13, checked
        offsets[i] = 0
        types[i] = 13
    elif np.all(vs==[ 1,-1, 0]) and np.all(Es==[1, 1, 0, 1]): # situation 14, didn't find any yet
        offsets[i] = 0
        types[i] = 14
    elif np.all(vs==[ 1, 0,-1]) and np.all(Es==[1, 1, 0, 1]): # situation 15, didn't find any yet
        offsets[i] = 0
        types[i] = 15
    elif np.all(vs==[ 0, 0, 1]) and np.all(Es==[1, 0, 1, 1]): # situation 16, checked
        offsets[i] = 0
        types[i] = 16
    elif np.all(vs==[ 0,-1, 1]) and np.all(Es==[1, 0, 1, 1]): # situation 17, did't find any yet
        offsets[i] = 0
        types[i] = 17
    elif np.all(vs==[-1, 0, 1]) and np.all(Es==[1, 0, 1, 1]): # situation 18, didn't find any yet
        offsets[i] = 0
        types[i] = 18
    elif np.all(vs==[ 1, 0, 0]) and np.all(Es==[1, 0, 1, 1]): # situation 19, didn't find any (check 187?)
        offsets[i] = 1
        types[i] = 19
    elif np.all(vs==[ 1,-1, 0]) and np.all(Es==[1, 0, 1, 1]): # situation 20, didn't find any (check 187?)
        offsets[i] = 1
        types[i] = 20
    elif np.all(vs==[ 1, 0,-1]) and np.all(Es==[1, 0, 1, 1]): # situation 21, didn't find any (check 187?)
        offsets[i] = 1
        types[i] = 21
    elif np.all(vs==[ 0, 0, 1]) and np.all(Es==[1, 1, 0, 1]): # situation 22, checked (rare)
        offsets[i] = -1
        types[i] = 22
    elif np.all(vs==[ 0,-1, 1]) and np.all(Es==[1, 1, 0, 1]): # situation 23, didn't find any
        offsets[i] = -1
        types[i] = 23
    elif np.all(vs==[-1, 0, 1]) and np.all(Es==[1, 1, 0, 1]): # situation 24, didn't find any yet
        offsets[i] = -1
        types[i] = 24
    else:
        offsets[i] = 2
        types[i] = 0
        
    if types[i]==923424:
        print(vs, Es)
        plt.figure()
        d = 20
        plt.plot(vls[idx-d:idx+d+1, 1000:1500].mean(1) - vls[idx-d:idx+d+1, 250:750].mean(1) - 10 * badvls[idx-d:idx+d+1], 'k.-')
        plt.plot(gmd[idx-d:idx+d+1]*100, 'b.')
        plt.plot(np.arange(2*d+1)*0 + 5, 'c--', alpha=0.6)
        plt.plot(np.arange(2*d+1)*0 + v_thresh, 'k--', alpha=0.6)
        plt.ylim(-10, 30)
        found +=1
    #if found>5:
    #    break

In [ ]:
plt.plot(*np.unique(offsets, return_counts=True), 'ro-')
plt.yscale('log')

In [ ]:
plt.plot(*np.unique(types, return_counts=True), 'ro-')
plt.yscale('log')
plt.grid(which='both')

In [ ]:
len(bykik_where), len(offsets), len(badvls_where)

In [ ]:
vls_timestamps = np.zeros_like(timestamps) - 9999
vls_fixtype = np.zeros_like(timestamps) - 9999
for bykik_0, bykik_1, offset_0, offset_1, type_0, type_1 in zip(bykik_where[:-1], bykik_where[1:], offsets[:-1], offsets[1:], types[:-1], types[1:]):
    
    badvls_shots = badvls_where[(bykik_0<=badvls_where)&(badvls_where<=bykik_1)]
    nbad = len(badvls_shots)
    if nbad not in [0, 1]:
        print(bykik_0)
        raise ValueError
    elif nbad==1:
        badvls_shot = badvls_shots[0]
        
    if offset_0==2 or offset_1==2:
        vls_fixtype[bykik_0:bykik_1+1] = -1 # couldn't identify the offset at limits
    elif offset_0==0 and offset_1==-1 and nbad==0:
        vls_fixtype[bykik_0:bykik_1+1] = -2 # offset got changed between these
    elif offset_0==0 and offset_1==-1 and nbad==1:
        if bykik_1 != badvls_shot:
            print("Weird scenario, ??:", bykik_0, offset_0, type_0, bykik_1, offset_1, type_1, badvls_shot)
        else:
            vls_fixtype[bykik_0:bykik_1+1] = -3 # broken, fixed, but immediately offset
    elif offset_0==0 and offset_1==0 and nbad==0:
        vls_timestamps[bykik_0:bykik_1+1] = timestamps[bykik_0:bykik_1+1]
        vls_fixtype[bykik_0:bykik_1+1] = 1
    elif offset_0==-1 and offset_1==-1 and nbad==0:
        vls_timestamps[bykik_0:bykik_1+1] = timestamps[bykik_0-1:bykik_1]
        vls_fixtype[bykik_0:bykik_1+1] = 2
    elif offset_0==-1 and offset_1==0 and nbad==1:
        vls_timestamps[bykik_0:badvls_shot+1] = timestamps[bykik_0-1:badvls_shot]
        vls_fixtype[bykik_0:badvls_shot+1] = 3
        vls_timestamps[badvls_shot+1:bykik_1+1] = timestamps[badvls_shot+1:bykik_1+1]
        vls_fixtype[badvls_shot+1:bykik_1+1] = 4
    elif offset_0==-1 and offset_1==-1 and nbad==1:
        vls_timestamps[bykik_0:badvls_shot+1] = timestamps[bykik_0-1:badvls_shot]
        vls_fixtype[bykik_0:badvls_shot+1] = 5
    elif offset_0==0 and offset_1==0 and nbad==1:
        vls_timestamps[badvls_shot+1:bykik_1+1] = timestamps[badvls_shot+1:bykik_1+1]
        vls_fixtype[badvls_shot+1:bykik_1+1] = 6
    elif offset_0==-1 and offset_1==0 and nbad==0:
        print("Weird scenario, fixed without None VLS:", bykik_0, offset_0, type_0, bykik_1, offset_1, type_1, badvls_shot)
    else:
        print("Unaccounted: ", bykik_0, offset_0, type_0, bykik_1, offset_1, type_1, badvls_shot)

In [ ]:
print(np.unique(vls_fixtype, return_counts=True)[0])
print(np.unique(vls_fixtype, return_counts=True)[1])

In [ ]:
idx = 44654
d = 30
plt.figure(figsize=(20, 5))
plt.plot(vls[:,1000:1500].mean(1) - vls[:,250:750].mean(1) - 10*badvls, 'k.-')
plt.plot(gmd*100, 'b.')
plt.plot(gmd*-99999 * (gmd>-100), 'rx')
plt.xlim(idx-d, idx+d+13)
plt.ylim(-15, 30)

In [ ]:
valid_offset = vls_fixtype > 0
assert np.all((timestamps==vls_timestamps[valid_offset][:,None]).sum(1)==1)
vls_fix_idx = (timestamps==vls_timestamps[valid_offset][:,None]).argmax(1)

In [ ]:
x, y = l3[valid_offset], CoM[valid_offset]
filt = ~np.isnan(y)
x, y = x[filt], y[filt]
plt.hist2d(x, y, bins=(np.linspace(5650, 5750, 100), np.linspace(1180, 1360, 110)));

In [ ]:
x, y = l3[valid_offset], CoM[vls_fix_idx]
filt = ~np.isnan(y)
x, y = x[filt], y[filt]
plt.hist2d(x, y, bins=(np.linspace(5650, 5750, 100), np.linspace(1180, 1360, 110)));

In [ ]:
plt.hist2d(gmd[valid_offset], vls[vls_fix_idx,1000:1500].mean(1)-vls[vls_fix_idx,250:750].mean(1), bins=(np.linspace(-0.05, 0.3, 100), np.linspace(0, 50)));

In [ ]:
plt.hist2d(gmd[valid_offset], vls[valid_offset,1000:1500].mean(1)-vls[valid_offset,250:750].mean(1), bins=(np.linspace(-0.05, 0.3, 100), np.linspace(0, 50)));

In [ ]:
plt.hist2d(gmd, vls[:,1000:1500].mean(1)-vls[:,250:750].mean(1), bins=(np.linspace(-0.05, 0.3, 100), np.linspace(0, 50)));

In [ ]:
for i in range(1, 7):
    plt.figure()
    assert np.all((timestamps==vls_timestamps[vls_fixtype==i][:,None]).sum(1)==1)
    vls_fix_idx = (timestamps==vls_timestamps[vls_fixtype==i][:,None]).argmax(1)
    plt.hist(timestamps[vls_fixtype==i] - timestamps[vls_fix_idx], bins=50);
    plt.yscale('log')

In [ ]:
idx = 47173
d = 40
plt.figure(figsize=(20,5))
plt.plot(vls[idx-d:idx+d+1, 1000:1500].mean(1) - vls[idx-d:idx+d+1, 250:750].mean(1) - 10 * badvls[idx-d:idx+d+1], 'k.-')
plt.plot(gmd[idx-d:idx+d+1]*100, 'b.')
plt.plot(gmd[idx-d:idx+d+1]*-99999 * (gmd[idx-d:idx+d+1]>-100), 'rx')
plt.plot(np.arange(2*d+1)*0 + 5, 'c--', alpha=0.6)
plt.plot(np.arange(2*d+1)*0 + v_thresh, 'k--', alpha=0.6)
plt.ylim(-10, 30)

In [ ]:
vls_timestamps

In [ ]:
print(vs)
np.digitize(vs, bins=[-99999, -15, v_thresh, 99999]) - 2

In [ ]:
plt.plot()

In [ ]:
    if (vs > v_thresh).sum()==3: # all high, very strange indeed
        offset = np.nan
    elif (vs > v_thresh).sum()==2: # two high, one low, this is the 'normal' scenario
        offset =  - vs.argmin() + 1
    elif (vs > v_thresh).sum()==1: # one high, two
        
    if vs[0] > v_thresh and vs[1] > v_thresh and vs[2] > v_thresh:
        offset = np.nan
    elif vs[0] <= v_thresh and vs[1] > v_thresh and vs[2] > v_thresh:
        offset = -1
    elif vs[0] > v_thresh and vs[1] <= v_thresh and vs[2] > v_thresh:
        offset = 0
    elif vs[0] > v_thresh and vs[1] > v_thresh and vs[2] <= v_thresh:
        offset = 1
    elif 

In [ ]:
print(vs)
print((Es*1e3).astype(int))

In [ ]:
for offset in [-1, 0, 1]:
    print(Es[1-offset:4-offset] > 0.05, vs > v_thresh)
    print(np.all((Es[1-offset:4-offset] > 0.05) == (vs > v_thresh)))

In [ ]:
v_thresh = 1.5
if vs[0] <= v_thresh and vs[1] > v_thresh and vs[2] > v_thresh:
    offset = -1
elif vs[0] > v_thresh and vs[1] <= v_thresh and vs[2] > v_thresh:
    offset = 0
elif vs[0] > v_thresh and vs[1] > v_thresh and vs[2] <= v_thresh:
    offset = 1

In [ ]:
plt.figure(dpi=120)
plt.hist(data, bins=np.linspace(-3, 80, 520), histtype='step');
plt.yscale('log')
plt.xlim(-3, 5)
plt.legend(offsets)

In [ ]:
plt.figure(dpi=120)
plt.hist2d(np.abs(data[:,[0, 1, 4, 5, 6]]).min(1), np.abs(data[:,2:4]).min(1), bins=np.linspace(-3, 80, 220), norm=LogNorm());
plt.axis('square')
plt.xlim(-3, 3)
plt.ylim(-3, 3)

In [ ]:
plt.figure(dpi=120)
plt.hist2d(data[:,4], np.abs(data[:,2:4]).min(1), bins=np.linspace(-3, 80, 220), norm=LogNorm());
plt.axis('square')
#plt.xlim(-3, 3)
#plt.ylim(-3, 3)

In [ ]:
plus_one = bykik_where[(np.abs(data[:,2])<1.5)&(np.abs(data[:,:2]).min(1)>1.5)]
print(plus_one)

In [ ]:
data.shape

In [ ]:
# where offset = -1 probably
np.random.choice(bykik_where[np.abs(data[:,2])<1.5], 5)

In [ ]:
f

In [ ]:
from scipy.ndimage.filters import gaussian_filter
def nan_smooth(U, sig=2, trunc=4):
    V = U.copy()
    V[np.isnan(U)]=0
    VV = gaussian_filter(V, sigma=sig, truncate=trunc)
    
    W = 0*U.copy() + 1
    W[np.isnan(U)] = 0
    WW = gaussian_filter(W, sigma=sig, truncate=trunc)
    
    return VV/WW
    
#plt.plot(gmd)
gmd_smooth = gmd.copy()
gmd_smooth[gmd<-1] = np.nan
gmd_smooth = nan_smooth(gmd_smooth, sig=4)
x = np.arange(len(gmd_smooth))
plt.figure(figsize=(25, 4))
plt.plot(x, gmd_smooth)

plt.plot(x[gmd_smooth<0.04], gmd_smooth[gmd_smooth<0.04], 'r.')
plt.ylim(-0.1, 0.5)
#plt.xlim(0, 100)
#plt.xlim(0, 18000)
#plt.xlim(2000, 7000)
#plt.xticks(np.arange(2000, 7001, 100));
plt.grid()

In [ ]:
plt.plot(badvls)
plt.xlim(0, 4500)

In [ ]:
plt.hist(nan_smooth(gmd_smooth, sig=2), bins=100);

In [ ]:
# where offset = 0 probably
np.random.choice(bykik_where[np.abs(data[:,3])<1.5], 5)

In [ ]:
# Can't tell!
both_low = bykik_where[np.abs(data[:,1:]).max(1)<1.5]
print(both_low)
print(gmd[both_low+1])

In [ ]:
# Can't tell!
both_high = bykik_where[np.abs(data[:,:]).min(1)>1.5]
print(both_high)

In [ ]:
plt.hist(gmd[both_low+1], bins=np.linspace(-0.1, 0.1, 10), histtype='step')
plt.hist(gmd[both_low+2], bins=np.linspace(-0.1, 0.1, 10), histtype='step')
plt.hist(gmd[both_low+3], bins=np.linspace(-0.1, 0.1, 10), histtype='step')

In [ ]:
# weird?
weird = bykik_where[(np.abs(data[:,2:4]).min(1)<1.5)&(np.abs(data[:,[0, 1, 4, 5, 6]]).min(1)<1.5)&(gmd_smooth[bykik_where]>0.1)]
print(weird)

In [ ]:
x0 = 51303 # shows a np.diff(bykik)==12 jump, offset of 0 probably
x0 = 39443 # can't tell between -1 and 0
x0 = 16571 # -1 offset probably

In [ ]:
badvls_where

In [ ]:

plt.figure(figsize=(25,4))
x0, d = 5000, 80
plotdata = vls[x0-d:x0+d+1,1000:1500].mean(1) - vls[x0-d:x0+d+1,250:750].mean(1) - 10 * badvls[x0-d:x0+d+1]
bykik_where_local = bykik_where[np.abs(bykik_where-x0)<d+1]
plotdata2 = vls[bykik_where_local][:,1000:1500].mean(1) - vls[bykik_where_local][:,250:750].mean(1)
phase = np.abs(plotdata).argmin() % 13
plt.plot(plotdata, 'k.-')
plt.plot(bykik_where_local-x0+d, plotdata2, 'rx')
plt.xticks(np.arange(d*2+1)[phase::13])
plt.grid()
#plt.gca().twinx()
gmddata = gmd[x0-d:x0+d+1]
plt.plot(gmddata * np.abs(np.median(plotdata)) / np.abs(np.median(gmddata)), 'b.')
plt.ylim(-10, 40)

In [ ]:
plt.plot(gmd[:100])
plt.ylim(-0.5, 0.5)
idx_ = np.arange(100)
gmd_ = gmd[:100]
vls_idx = [0, 1, 2, 3, 4, -999,-999,-999, 9, 10, 11, -999, 12, ]
vls_ = vls[vls_idx]

In [ ]:
vls.shape, timestamps.shape

In [ ]:
plt.hist(np.diff(timestamps)/1e6, bins=100);#bins=np.linspace(0.5, 5000, 100));
plt.yscale('log')

In [ ]:
vls.shape

In [ ]:
badvls_where

In [ ]:
plt.plot(data);
plt.xlim(0, 100)
plt.ylim(-3, 80)

In [ ]:
(data==-9999).sum(0)

In [ ]:
data = []
for offset in [-3, -2, -1, 0, 1, 2, 3]:
    data.append(vls[lim+offset:-lim+offset, 1000:1500].mean(1))
    data.append(vls[5+offset:-5+offset,1000:1500].mean(1)[~xray[5:-5]] - vls[10+offset:-10+offset,0:500].mean(1)[~xray[10:-10]])
data = np.array(data)

In [ ]:
plt.figure(dpi=120)
plt.hist2d(data[5][newb], data[2][newbad], bins=np.linspace(-20, 80, 120), norm=LogNorm());
plt.axis('square')

In [ ]:
[k for k in raw.keys() if 'las' in k]
delay_tgt = raw['epics_las_fs14_target_time'][:][order]
plt.plot(ts*1e-3, delay_tgt)
plt.xlim

In [ ]:
for xlow, xhigh in [(0, 50), (0, 1000), (1000, 2000), (2000, 3000), (3000, 3800)]:
    plt.figure(figsize=(20,4))
    plt.plot(ts[:-1]*1e-3, np.diff(timestamps), 'r.-')
    #plt.xlim(0, 10)
    plt.xlim(xlow, xhigh)
    #plt.xlim(8000, 9000)
    plt.yscale('log')
    plt.plot(ts*1e-3, badvls*1e3+1, 'b.-')
    plt.plot(ts*1e-3, (delay_tgt - delay_tgt.min())*1e8 + 1e3, 'k')
    
    plt.plot(ts[10:-10][~xray[10:-10]]*1e-3, np.array(data).argmin(0)*1e6 + 3e3, 'g.-')

In [ ]:
for xlow, xhigh in [(0, 50), (2000, 2030), (0, 1000), (1000, 2000), (2000, 3000), (3000, 3800)]:
    plt.figure(figsize=(20,4))
    plt.plot(ts[:-1]*1e-3, np.diff(timestamps) > 10e6, 'r.-')
    #plt.xlim(0, 10)
    plt.xlim(xlow, xhigh)
    #plt.xlim(8000, 9000)
    #plt.yscale('log')
    plt.plot(ts*1e-3, badvls, 'b.-')
    #plt.plot(ts*1e-3, (delay_tgt - delay_tgt.min())*1e8 + 1e3, 'k')
    
    plt.plot(ts[10:-10][~xray[10:-10]]*1e-3, np.array(data).argmin(0), 'g.-')

In [ ]:
ts.shape, xray.shape, data[0].shape, (~xray[10:-10]).sum()

In [ ]:
ts[-10:10][(~xray)[10:-10]].shape

In [ ]:
plt.hist2d(badvls[:-1], np.diff(timestamps), bins=(2, 2), norm=LogNorm())
plt.colorbar()


In [ ]:
plt.plot(badvls[:-1]==(np.diff(timestamps)>100e6))

In [ ]:
badvls.shape, timestamps.shape

In [ ]:
plt.plot((np.diff(ts)*1e-3)[:40])

In [ ]:
diff_ts = np.diff(ts)
where_big = np.where(diff_ts>50)[0]
plt.plot(np.diff(where_big))

In [ ]:
plt.hist(diff_ts[where_big], bins=100);

In [ ]:
plt.hist(np.diff(ts)*1e-3, bins=200);

In [ ]:
(ts[badvls]*1e-3)[:10]

In [ ]:
plt.plot(np.diff(ts[badvls]*1e-3))

In [ ]:
plt.figure(figsize=(25,3))
plt.plot(np.array(data).argmin(0) - 1, '.-')
plt.plot(gaussian_filter(np.array(data).argmin(0).astype(float) - 1, 30), 'r')
#plt.xlim(6500, 6600)
plt.figure(figsize=(25,3))
plt.plot(np.array(data).argmin(0) - 1, '.-')
plt.xlim(0, 1200)
plt.figure(figsize=(25,3))
plt.plot(np.array(data).argmin(0) - 1, '.-')
plt.xlim(3000, 4200)

In [ ]:
plt.figure(figsize=(25,3))
plt.plot(data[0] - data[1], '.-')
#plt.xlim(6500, 6600)
plt.ylim(-15,15)
plt.grid()
plt.figure(figsize=(25,3))
plt.plot(data[0] - data[1], '.-')
plt.xlim(0, 1000)
plt.ylim(-15,15)
plt.grid()

In [ ]:
plt.hist2d(np.abs(resids[1]), np.abs(resids[2]), bins=np.linspace(0, 25, 100));

In [ ]:
plt.figure(figsize=(25,5))
plt.plot(np.polyval( np.polyfit(x, y, 1), x) - y)
plt.xlim(0, 300)

In [ ]:
np.corrcoef(x, y)

In [ ]:
CoM[:-off][filt[:-off]].shape, l3[off:][filt[off:]].shape

In [ ]:
plt.figure(figsize=(15,8))
index = 1
nbins = 90
filt = ~good# & (np.arange(len(gmd)) > len(gmd)/2) & (1320 < CoM)
for off in [2, 1]:
    plt.subplot(2,5,index)
    index+=1
    plt.hist2d(vls[filt,1200:1500].mean(1)[:-off], gmd[filt][off:], bins=nbins);
    plt.title("offset: %d" % -off)
    #plt.xlim(304, 350)
    plt.subplot(2,5,index+4)
    plt.hist2d(vls[filt,1200:1500].mean(1)[:-off], xgmd[filt][off:], bins=nbins);
    plt.title("offset: %d" % -off)
    #plt.xlim(304, 350)
plt.subplot(2,5,index)
index+=1
plt.hist2d(vls[filt,1200:1500].mean(1), gmd[filt], bins=nbins)
plt.title("offset: 0")
#plt.xlim(304, 350)
plt.subplot(2,5,index+4)
plt.hist2d(vls[filt,1200:1500].mean(1), xgmd[filt], bins=nbins)
plt.title("offset: 0")
#plt.xlim(304, 350)
for off in [1, 2]:
    plt.subplot(2,5,index)
    index+=1
    plt.hist2d(vls[filt,1200:1500].mean(1)[off:], gmd[filt][:-off], bins=nbins);
    plt.title("offset: %d" % off)
    #plt.xlim(304, 350)
    plt.subplot(2,5,index+4)
    plt.hist2d(vls[filt,1200:1500].mean(1)[off:], xgmd[filt][:-off], bins=nbins);
    plt.title("offset: %d" % off)
    #plt.xlim(304, 350)
plt.tight_layout()

In [ ]:
timetool = raw['tmo_atmopal_fltpos_ps'][:][order]
#timetool = raw['tmo_atmopal_fltpos'][:][order]

In [ ]:
delays

In [ ]:
plt.figure(figsize=(15,8))
delays_ps = (delays.mean() - delays) * 1e6
for i, d in enumerate(delays[2:7]):
    plt.hist(timetool[i+2==delay_idx], bins=100, alpha=0.7, label=d);
#plt.yscale('log')
#plt.xlim(-0.6, 0)
plt.legend()
plt.ylim(0, 500)

In [ ]:
valid_tt = (1<delay_idx)&(delay_idx<7)&(np.abs(timetool)>0.001) & (timetool<0.8)
timetools, timetool_idx = np.unique(np.around(timetool, 3), return_inverse=True)
plt.hist(timetools[timetool_idx[valid_tt]], bins=np.linspace(-1, 1, 150));
#plt.ylim(0, 30)
#plt.xlim(-1, 1.2)
#plt.xlim(-0.005, 0.005)

In [ ]:
Nbins_L3 = 50
cnts_L3, bins_L3, _ = plt.hist(l3[good], bins=np.linspace(5640, 5750, Nbins_L3+1), log=True)
idx_L3 = np.digitize(l3, bins_L3) - 1
good = good & (0<=idx_L3) & (idx_L3<Nbins_L3)
bins_L3 = (bins_L3[:-1] + bins_L3[1:]) / 2
plt.xlabel("L3 energy")
plt.ylabel("# shots")

In [ ]:
shots_per_L3 =  np.bincount(idx_L3[good], minlength=Nbins_L3)
gmd_vs_L3 = np.bincount(idx_L3[good], weights=gmd[good], minlength=Nbins_L3) / shots_per_L3

In [ ]:
plt.hist2d(l3[good], gmd[good], bins=200, norm=LogNorm());
plt.ylim(0,)
plt.xlabel("L3 energy")
plt.ylabel("GMD energy")
plt.plot(bins_L3, gmd_vs_L3, 'r')
plt.ylim(0,)
plt.gca().twinx()
plt.plot(bins_L3, shots_per_L3, 'k')
#plt.yscale('log')
plt.ylabel("# shots", rotation=270)
plt.ylim(0,)

In [ ]:
plt.hist2d(l3[good], CoM[good], bins=200, norm=LogNorm());


In [ ]:
plt.figure(figsize=(25,4))
plt.plot(m_q, wfs[good&jet&xray&goose].mean(0))
plt.plot(m_q, wfs[good&jet&xray&~goose].mean(0))
plt.plot(m_q, wfs[jet&~xray&~goose].mean(0))
plt.xlim(0, 105)
plt.xticks(np.arange(0, 105.1, 1), rotation=90)
plt.grid()
plt.ylim(-20, 30)

In [ ]:
mpks = [15, 27, 29, 37, 43, 69, 85, 100]
XAS = np.zeros((len(mpks), 2, len(delays), len(bins_L3)))
for i, mpk in enumerate(mpks):
    tof_filt = (mpk - 0.25 < m_q) & (m_q < mpk + 0.25)
    for j in [0, 1]:
        for k, d in enumerate(delays):
            data_filt = good & jet & xray & (goose==(1-j)) & (delay_idx==k)
            xas = np.bincount(idx_L3[data_filt], weights=wfs[data_filt][:,tof_filt].mean(1), minlength=Nbins_L3)
            xas /= np.bincount(idx_L3[data_filt], weights=gmd[data_filt], minlength=Nbins_L3)
            XAS[i, j, k] = xas.copy()

In [ ]:
goose.sum(), len(goose)

In [ ]:
xas.shape

In [ ]:
print(delays)
jump = 10
plt.figure(figsize=(10, 6))
for i, mpk in enumerate(mpks):
    #if mpk!=43: continue
    
    
    y = XAS[i, 0, :].T
    plt.plot(bins_L3,  y / np.nanmax(y), label=mpk)
    #plt.plot(bins_L3, XAS[i, 1, :].T + np.arange(len(delays))*jump, 'r', alpha=0.8)
    #plt.yticks(np.arange(0, len(delays)*jump, jump), delays)
    #plt.title(mpk)
plt.grid()
plt.legend(ncol=2)
plt.ylim(-0.05, 1.05)
plt.xlabel("L3 energy / MeV")
plt.ylabel("")

In [ ]:
plt.figure(figsize=(20,10))
delays_ps = (delays.mean() - delays) * 1e6
for i, d in enumerate(delays):
    plt.hist(timetool[i==delay_idx], bins=np.linspace(0, 1000, 150), alpha=0.7, label=d);
#plt.yscale('log')
#plt.xlim(-0.6, 0)
plt.legend()
plt.ylim(0, 500)

In [ ]:
plt.plot(m_q, gaussian_filter(wfs[jet&~xray].mean(0), 3))
plt.plot(m_q, gaussian_filter(wfs[jet&xray].mean(0), 3))
plt.xlim(0, 110)
plt.ylim(-2, 2)

In [ ]:
plt.figure(figsize=(20,4))
plt.plot(m_q, wfs[jet&xray&~goose&(delay>5438.231)&(5695<l3)&(l3<5702)].mean(0), label='ontime high delay')
plt.plot(m_q, wfs[jet&xray&~goose&(delay<=5438.231)&(5695<l3)&(l3<5702)].mean(0), label='ontime low delay')
plt.plot(m_q, wfs[jet&xray&goose&(delay>5438.231)&(5695<l3)&(l3<5702)].mean(0), label='goose high delay')
plt.plot(m_q, wfs[jet&xray&goose&(delay<=5438.231)&(5695<l3)&(l3<5702)].mean(0), label='goose low delay')
plt.xlim(0, 110)
plt.ylim(-3, 4)
plt.xticks(np.arange(0, 120.1, 1))
plt.legend()
plt.xlim(20, 60)
plt.grid()
plt.xlim(41, 45)

In [ ]:
delays